To crea opencv python environment:
```
conda create --name py35-opencv3 python=3.5
activate py35-opencv3
conda install numpy
conda install anaconda-client
conda install --channel https://conda.anaconda.org/menpo opencv3
conda install jupyter
```

In [48]:
# import
import pandas as pd
import cv2
import numpy as np

In [53]:
video_file = 'D:/ProblemaCamTacticaLugo/3245-scoutingvideo-1.mp4'
calib_file = 'D:/ProblemaCamTacticaLugo/calib.csv'

# sport field
f_tl = [-50.0, 32.0] 
f_tr = [50.0, 32.0]
f_bl = [-50.0, -32.0] 
f_br = [50.0, -32.0]

In [54]:
def homography_pitch_to_image(P):
    """
    Returns the homografy to transform points from the pitch (world Z=0) to texture image
    :param matrixP: Full matrix P
    :return: homography_pitch_to_texture matrix
    """
    # delete the 3th column, the z component
    return np.delete(P, 2, axis=1)

def pitch_to_image(pitch, H):
    """
    Project a point from pitch plane to texture plane
    :param pitch: pitch point (x, y)
    :param H: projection matrix P
    :return: texture point (i, j)
    """
    pitch_h = np.append(pitch, 1.0)
    transformed = H.dot(pitch_h)
    return np.array([transformed[0]/transformed[2], transformed[1]/transformed[2]])



In [59]:
# open video
# Create a VideoCapture object and read from input file
# If the input is the camera, pass 0 instead of the video file name
cap = cv2.VideoCapture(video_file)
 
# Check if camera opened successfully
if (cap.isOpened()== False): 
  print("Error opening video stream or file")
 
# open atv calibration csv file
calib = pd.read_csv(calib_file)

counter = 0
for index, row in calib.iterrows():   
    # get video frame
    ret, frame = cap.read()
    counter += 1
    if counter < 25*100:
        continue
    if not ret:
        print('No more video frames')
        break
        
    # get matrix P
    P = np.reshape(row.as_matrix(), (3, 4))
    # get Homography to project pitch points to video image
    H = homography_pitch_to_image(P)
    
    # project point to image
    p_center = [0.0, 0.0]
    p_center_img = pitch_to_image(p_center, H).astype(int)
    cv2.circle(frame, tuple(p_center_img), 3, (0, 0, 255), -1)
    
    f_tl_img = tuple(pitch_to_image(f_tl, H).astype(int))
    f_tr_img = tuple(pitch_to_image(f_tr, H).astype(int))
    f_bl_img = tuple(pitch_to_image(f_bl, H).astype(int))
    f_br_img = tuple(pitch_to_image(f_br, H).astype(int))
    
    pts = np.array([f_bl_img, f_br_img, f_tr_img, f_tl_img], np.int32)
    pts = pts.reshape((-1, 1, 2))
    cv2.polylines(frame, [pts], False, (0, 255, 0))

    # Display the resulting frame
    cv2.imshow('Frame', frame)
    
    # Press Q on keyboard to  exit
    if cv2.waitKey(25) & 0xFF == ord('q'):
        break
 
# When everything done, release the video capture object
cap.release()
 
# Closes all the frames
cv2.destroyAllWindows()
